In [9]:
import pandas as pd
import os

In [10]:
# RENOMBRAR CCAA PARA PODER GRAFICAR EN CONJUNTO

mapeo_ccaa = {
    "com.madrid": "Madrid",
    "comunidad de madrid": "Madrid",
    "madrid": "Madrid",
    "com.valenciana": "Comunidad Valenciana",
    "comunidad valenciana": "Comunidad Valenciana",
    "comunitat valenciana": "Comunidad Valenciana",
    "valencia": "Comunidad Valenciana",
    "castilla la mancha": "Castilla - La Mancha",
    "comunidad foral de navarra": "Navarra",
    "c.f.navarra": "Navarra",
    "comunidad de navarra": "Navarra",
    "navarra": "Navarra",
    "baleares": "Illes Balears",
    "illes balears": "Illes Balears",
    "castilla y león": "Castilla y León",
    "cataluña": "Cataluña",
    "principado de asturias": "Principado de Asturias",
    "asturias": "Principado de Asturias",
    "región de murcia": "Región de Murcia",
    "murcia": "Región de Murcia",
    "país vasco": "País Vasco",
    "la rioja": "La Rioja",
    "ceuta": "Ceuta",
    "melilla": "Melilla",
    "transito": None}

In [11]:
def renombrar_ccaa(texto):
    if pd.isna(texto):
        return texto
    texto = texto.strip().lower()
    return mapeo_ccaa.get(texto, texto.title())

In [12]:
# CÓDIGO REUTILIZABLE

def cargar_csv(nombre_archivo):
    ruta = os.path.join("./data", nombre_archivo)
    return pd.read_excel(ruta)

In [13]:
def limpiar(df, columna_ccaa, columnas_num=None):
    df = df.copy()
    df[columna_ccaa] = df[columna_ccaa].apply(renombrar_ccaa)
    if columnas_num:
        for col in columnas_num:
            if col in df.columns:
                df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

In [14]:
# CARGAR 

df_internacional = pd.read_excel("./data/FRONTUR_DL.xlsx")
df_nacional = pd.read_excel("./data/TURISMO_INTERNO_PROV_CCAA_DL.xlsx")
df_gasto = pd.read_excel("./data/GASTO_TPV_DESTINO_CCAA_MES_DL.xlsx")

In [15]:
# LIMPIAR Y RENOMBRAR

df_internacional = limpiar(df_internacional, "CCAA_DESTINO", ["VISITANTES"])
df_nacional = limpiar(df_nacional, "CCAA_DESTINO", ["TURISTAS"])
df_gasto = limpiar(df_gasto, "CCAA_DESTINO", ["GASTO"])

In [16]:
# ELIMINAR FILAS CON CCAA None

df_internacional = df_internacional[df_internacional["CCAA_DESTINO"].notna()]
df_nacional = df_nacional[df_nacional["CCAA_DESTINO"].notna()]
df_gasto = df_gasto[df_gasto["CCAA_DESTINO"].notna()]

In [17]:
# FILTRAR AÑOS PARA QUE COINCIDAN A PARTIR DE 2019

df_internacional = df_internacional[df_internacional["AÑO"] >= 2019]
df_nacional = df_nacional[df_nacional["AÑO"] >= 2019]
df_gasto = df_gasto[df_gasto["AÑO"] >= 2019]

In [18]:
# AGRUPAR SUMANDO

df_internacional_sumado = df_internacional.groupby(["AÑO", "CCAA_DESTINO"], as_index=False)["VISITANTES"].sum()
df_nacional_sumado = df_nacional.groupby(["AÑO", "CCAA_DESTINO"], as_index=False)["TURISTAS"].sum()
df_gasto_sumado = df_gasto.groupby(["AÑO", "CCAA_DESTINO"], as_index=False)["GASTO"].sum()

In [19]:
# RENOMBRAR COLUMNAS

df_internacional_sumado.rename(columns={"AÑO": "Año", "CCAA_DESTINO": "CCAA", "VISITANTES": "Turistas_Internacionales"}, inplace=True)
df_nacional_sumado.rename(columns={"AÑO": "Año", "CCAA_DESTINO": "CCAA", "TURISTAS": "Turistas_Nacionales"}, inplace=True)
df_gasto_sumado.rename(columns={"AÑO": "Año", "CCAA_DESTINO": "CCAA", "GASTO": "Gasto"}, inplace=True)

In [20]:
# MERGE DATAFRAMES

df_final = df_internacional_sumado.merge(df_nacional_sumado, how="outer", on=["Año", "CCAA"]).merge(df_gasto_sumado, how="outer", on=["Año", "CCAA"])

In [21]:
# NaNs POR 0

df_final.fillna(0, inplace=True)

In [22]:
df_final

,Año,CCAA,Turistas_Internacionales,Turistas_Nacionales,Gasto
0,2019,Andalucía,14652617.0,40609516.0,2.502729e+09
1,2019,Aragón,853190.0,8464283.0,3.744974e+08
2,2019,Canarias,13860630.0,3594802.0,7.364008e+08
3,2019,Cantabria,419761.0,4738769.0,1.852133e+08
4,2019,Castilla - La Mancha,264722.0,17516749.0,6.144751e+08
...,...,...,...,...,...
128,2025,Melilla,589.0,44345.0,5.890543e+06
129,2025,Navarra,1567395.0,1053054.0,1.005016e+08
130,2025,País Vasco,4603828.0,2927097.0,2.937720e+08
131,2025,Principado de Asturias,93765.0,1218067.0,1.076262e+08


In [23]:
def invertir_columnas(df, año):
    df_año = df[df["Año"] == año].copy()
    print(f"{len(df_año)} filas para el año {año}")
    return df_año.set_index("CCAA")[["Turistas_Internacionales", "Turistas_Nacionales", "Gasto"]].T

In [24]:
df_reorganizado = invertir_columnas(df_final, 2024)
df_reorganizado

19 filas para el año 2024


CCAA,Andalucía,Aragón,Canarias,Cantabria,Castilla - La Mancha,Castilla y León,Cataluña,Ceuta,Comunidad Valenciana,Extremadura,Galicia,Illes Balears,La Rioja,Madrid,Melilla,Navarra,País Vasco,Principado de Asturias,Región de Murcia
Turistas_Internacionales,1.634321e+07,7.128510e+05,1.603649e+07,5.308710e+05,2.366460e+05,2.007017e+06,2.843456e+07,64048.00,1.231256e+07,1.368929e+06,4.542561e+06,1.568414e+07,1.603070e+05,8.886161e+06,29078.00,3.811245e+06,1.042985e+07,3.407150e+05,1.179558e+06
Turistas_Nacionales,7.206146e+07,1.591372e+07,7.246561e+06,8.582125e+06,2.976146e+07,3.703512e+07,5.628166e+07,367856.00,3.777148e+07,9.866701e+06,2.161629e+07,5.809212e+06,4.217006e+06,4.114421e+07,237701.00,5.504122e+06,1.398123e+07,7.049120e+06,7.142906e+06
Gasto,4.071951e+09,5.859975e+08,1.308736e+09,3.089941e+08,1.001943e+09,1.395073e+09,5.439457e+09,23096660.05,2.910655e+09,2.392474e+08,6.824462e+08,2.012715e+09,2.081504e+08,3.799204e+09,6450267.42,5.244533e+08,7.342943e+08,2.042972e+08,7.191382e+08


In [25]:
# GUARDAR LOS DATAFRAMES DE INTERÉS

df_final.to_csv("./data/df_final.csv", index=False)
df_reorganizado.to_csv("./data/df_reorganizado.csv")